In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
a, b, c = np.meshgrid(np.arange(1, 7), np.arange(1, 7), np.arange(1, 7))

sample_space = np.array([a.ravel(), b.ravel(), c.ravel()]).T

In [4]:
def one(n):
  payout = -1 * np.ones(216)

  payout[(sample_space == n).sum(axis = 1) == 1] += (1 + 1)
  payout[(sample_space == n).sum(axis = 1) == 2] += (1 + 2)
  payout[(sample_space == n).sum(axis = 1) == 3] += (1 + 12)

  return payout

def double(n):
  payout = -1 * np.ones(216)
  payout[np.sum(sample_space == n, axis = 1) >= 2] += (1 + 12)

  return payout

def triple(n):
  payout = -1 * np.ones(216)
  payout[np.all(sample_space == n, axis = 1)] += (1 + 190)

  return payout

def all_triples():
  payout = -1 * np.ones(216)
  payout[np.all(sample_space == sample_space[:,0].reshape(-1, 1), axis = 1)] += (1 + 31)

  return payout

def big():
  payout = -1 * np.ones(216)
  payout[(sample_space.sum(axis = 1) >= 11) & (sample_space.sum(axis = 1) != 18)] += (1 + 1)

  return payout

def small():
  payout = -1 * np.ones(216)
  payout[(sample_space.sum(axis = 1) <= 10) & (sample_space.sum(axis = 1) != 3)] += (1 + 1)

  return payout

def two(n, m):
  payout = -1 * np.ones(216)
  payout[np.any(sample_space == n, axis = 1) & np.any(sample_space == m, axis = 1)] += (1 + 6)

  return payout

def sicbo_sum(s):
  sums = [62, 31, 18, 12, 8, 7, 6] # 4 and 17, 5 and 16, ...

  payout = -1 * np.ones(216)

  if s <= 10:
    payout[sample_space.sum(axis = 1) == s] += (1 + sums[s - 4])

  else:
    payout[sample_space.sum(axis = 1) == s] += (1 + sums[17 - s])

  return payout




## genetic algorithm
note there are 50 possible buttons to press (all triples takes up 2 spaces on the board)

In [80]:
def schedule(x):
  return 0.3 * (1 + np.exp(0.05 * (x - 50)))**(-1)

In [81]:
possibilities = [one(n) for n in range(1, 7)]
possibilities += [double(n) for n in range(1, 7)]
possibilities += [triple(n) for n in range(1, 7)]
possibilities += [all_triples(), big(), small()]
possibilities += [two(n, m) for n in range(1, 7) for m in range(n + 1, 7)]
possibilities += [sicbo_sum(i) for i in range(4, 18)]

possibilities = np.stack(possibilities)
possibilities.shape

(50, 216)

In [128]:
N = 400
B = 10

proposals = np.random.binomial(1, 0.4, size = (N, 50))
proposals = (proposals == 1)


for x in range(120):

  positives = []

  for i in range(N):

    payouts_i = possibilities[proposals[i]].sum(axis = 0)
    positives_i = (payouts_i > 0).sum()
    positives.append(positives_i)

  inds = np.argsort(positives)
  print('=== Epoch', x, '| Positives:', max(positives), '===')


  best = proposals[inds[-B:]]
  best_tile_aux = np.tile(best.T, int((N - B) / B)).T

  p = schedule(x)

  mutation = np.random.binomial(1, p, size = (N - B, 50))
  mutation = (mutation == 1)

  best_tile = np.bitwise_xor(mutation, best_tile_aux)
  proposals = np.vstack((best, best_tile))

=== Epoch 0 | Positives: 104 ===
=== Epoch 1 | Positives: 104 ===
=== Epoch 2 | Positives: 108 ===
=== Epoch 3 | Positives: 108 ===
=== Epoch 4 | Positives: 108 ===
=== Epoch 5 | Positives: 112 ===
=== Epoch 6 | Positives: 112 ===
=== Epoch 7 | Positives: 114 ===
=== Epoch 8 | Positives: 115 ===
=== Epoch 9 | Positives: 115 ===
=== Epoch 10 | Positives: 120 ===
=== Epoch 11 | Positives: 120 ===
=== Epoch 12 | Positives: 120 ===
=== Epoch 13 | Positives: 120 ===
=== Epoch 14 | Positives: 120 ===
=== Epoch 15 | Positives: 124 ===
=== Epoch 16 | Positives: 124 ===
=== Epoch 17 | Positives: 124 ===
=== Epoch 18 | Positives: 124 ===
=== Epoch 19 | Positives: 124 ===
=== Epoch 20 | Positives: 124 ===
=== Epoch 21 | Positives: 124 ===
=== Epoch 22 | Positives: 124 ===
=== Epoch 23 | Positives: 125 ===
=== Epoch 24 | Positives: 125 ===
=== Epoch 25 | Positives: 125 ===
=== Epoch 26 | Positives: 125 ===
=== Epoch 27 | Positives: 125 ===
=== Epoch 28 | Positives: 125 ===
=== Epoch 29 | Positives

In [129]:
possibilities_str = [f'one({n})' for n in range(1, 7)]
possibilities_str += [f'double({n})' for n in range(1, 7)]
possibilities_str += [f'triple({n})' for n in range(1, 7)]
possibilities_str += ['all_triples()', 'big()', 'small()']
possibilities_str += [f'two({n}, {m})' for n in range(1, 7) for m in range(n + 1, 7)]
possibilities_str += [f'sicbo_sum({i})' for i in range(4, 18)]

possibilities_str = np.array(possibilities_str)

possibilities_str[proposals[inds[-1]]]

array(['one(1)', 'one(2)', 'one(3)', 'one(4)', 'one(5)', 'double(2)',
       'double(3)', 'double(4)', 'double(5)', 'triple(2)', 'big()',
       'small()', 'two(1, 2)', 'two(1, 3)', 'two(1, 4)', 'two(1, 5)',
       'two(1, 6)', 'two(2, 3)', 'two(2, 4)', 'two(2, 5)', 'two(2, 6)',
       'two(3, 4)', 'two(3, 5)', 'two(3, 6)', 'two(4, 5)', 'two(4, 6)',
       'two(5, 6)'], dtype='<U13')

In [130]:
possibilities[proposals[inds[-1]]].sum(axis = 0)

array([-14., -13., -13., -13., -13., -15., -13.,   0.,   2.,   2.,   2.,
         0., -13.,   2.,   0.,   2.,   2.,   0., -13.,   2.,   2.,   0.,
         2.,   0., -13.,   2.,   2.,   2.,   0.,   0., -15.,   0.,   0.,
         0.,   0., -16., -13.,   0.,   2.,   2.,   2.,   0.,   0., 192.,
         0.,   0.,   0.,  -2.,   2.,   0.,   0.,   2.,   2.,   0.,   2.,
         0.,   2.,   0.,   2.,   0.,   2.,   0.,   2.,   2.,   0.,   0.,
         0.,  -2.,   0.,   0.,   0., -16., -13.,   2.,   0.,   2.,   2.,
         0.,   2.,   0.,   0.,   2.,   2.,   0.,   0.,   0.,   1.,   0.,
         0.,  -2.,   2.,   2.,   0.,   0.,   2.,   0.,   2.,   2.,   0.,
         2.,   0.,   0.,   0.,   0.,  -2.,   0.,   0., -16., -13.,   2.,
         2.,   0.,   2.,   0.,   2.,   0.,   2.,   0.,   2.,   0.,   2.,
         2.,   0.,   0.,   2.,   0.,   0.,   0.,   0.,   1.,   0.,  -2.,
         2.,   2.,   2.,   0.,   0.,   0.,   0.,   0.,   0.,  -2.,   0.,
       -16., -13.,   2.,   2.,   2.,   0.,   0.,   